In [ ]:
import pandas as pd

import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

# S3 prefix
prefix = 'Scikit-iris'

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [ ]:
sagemaker_session = sagemaker.Session()
input_train = sagemaker_session.upload_data(path='./data/train.csv', key_prefix='kaggle-ml-pipeline/data')
input_test = sagemaker_session.upload_data(path='./data/test.csv', key_prefix='kaggle-ml-pipeline/data')

In [ ]:
input_train

In [ ]:
!cat ./scripts/preprocess/Dockerfile

In [ ]:
!docker build -t sagemaker-kaggle-titanic-preprocess ./scripts/preprocess

In [ ]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

# SageMakerFullAccess を使っているから repository 名の中に sagemaker が含まれている必要がある
ecr_repository = f'sagemaker-kaggle-titanic-preprocess'
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository+tag}'

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t {ecr_repository} .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print(f'コンテナは {image_uri} へ登録されています。')

In [ ]:
image_uri == '815969174475.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-kaggle-titanic-preprocess'

In [ ]:
image_uri

In [ ]:
processing_input_dir = '/opt/ml/processing/input'
processing_output_dir = '/opt/ml/processing/output'
job_name = f'sagemaker-kaggle-preprocessing-train'

output_s3_path = 's3://' + sagemaker_session.default_bucket() + '/kaggle-ml-pipeline/preprocessed'

In [ ]:
output_s3_path

In [ ]:
processor = ScriptProcessor(base_job_name=job_name,
                                   image_uri=image_uri,
                                   command=['python3'],
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.c5.xlarge'
                                  )

In [ ]:
processor.run(code='./scripts/preprocess/preprocess_script/preprocess.py', # S3 の URI でも可
              inputs=[ProcessingInput(source=input_train, destination=processing_input_dir)],
              outputs=[ProcessingOutput(source=processing_output_dir, destination=output_s3_path)],
              arguments=[
                  '--data_type', 'train',
                  '--input_dir',processing_input_dir,
                  '--output_dir',processing_output_dir
                      ]
                    )

## 学習

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

sklearn = SKLearn(
    entry_point='scripts/train/train.py',
    framework_version='0.20.0',
    instance_type="ml.m5.xlarge",
    role=role)

In [ ]:
train_input = output_s3_path + '/train.csv'
sklearn.fit({'train': train_input})

### 推論

In [ ]:
processor.run(code='./scripts/preprocess/preprocess_script/preprocess.py', # S3 の URI でも可
              inputs=[ProcessingInput(source=input_test, destination=processing_input_dir)],
              outputs=[ProcessingOutput(source=processing_output_dir, destination=output_s3_path)],
              arguments=[
                  '--data_type', 'test',
                  '--input_dir',processing_input_dir,
                  '--output_dir',processing_output_dir
                      ]
                    )

In [ ]:
transformer = sklearn.transformer(instance_count=1, instance_type='ml.m5.xlarge')

In [ ]:
# Start a transform job and wait for it to finish
transformer.transform(batch_input_s3, content_type='text/csv')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()